In [ ]:
%run ../Helper/Setup.ipynb

In [ ]:
def import_csv(file, group=True):
    df = pd.read_csv(file)
    
    if group:
        df = group_data(df)
    
    df = df[~np.isnan(df.goodput)]
    
    return df

def group_data(df):
    filtered = df.groupby(by=['server', 'client'], as_index=False).agg({'goodput': ['mean', 'std'], 'repetition': 'count'})
    filtered.columns = list(map(''.join, filtered.columns.values))
    return filtered

In [ ]:
def extend_perf_client(x):
    columns = {}
    result = json.loads(x.replace("'", '"'))
    # sum_samples = result["SUM_SAMPLES_QUIC"]
    sum_samples = result["SUM_SAMPLES"]

    for key in result["OVERVIEW"].keys():
        # print(f"{key}: {result['OVERVIEW'][key]/sum_samples}")
        columns["client_"+key] = result['OVERVIEW'][key]/sum_samples*100
        #columns["client_"+key] = result['OVERVIEW'][key]
    return pd.Series(columns)
        
def extend_perf_server(x):
    columns = {}
    result = json.loads(x.replace("'", '"'))
    # sum_samples = result["SUM_SAMPLES_QUIC"]
    sum_samples = result["SUM_SAMPLES"]


    for key in result["OVERVIEW"].keys():
        # print(f"{key}: {result['OVERVIEW'][key]/sum_samples}")
        columns["server_"+key] = result['OVERVIEW'][key]/sum_samples*100
        # columns["server_"+key] = result['OVERVIEW'][key]
    return pd.Series(columns)

In [ ]:
def plot_perf(df, host):
    implementations = pd.unique(df[['server', 'client']].values.ravel('K'))
    categories = ['Uncategorized', 'Packet I/O', 'I/O', 'Crypto', 'Conn. Mgmt',]
    categories_data = ['GENERAL', 'PACKET_IO', 'IO', 'CRYPTO', 'CONN_MGMT',]

    lable = [host + "_" + x for x in categories_data]
    
    implementation_order = {
        'lsquic': 1,
        'quiche': 2,
    }
    
    implementations = sorted(implementations, key=lambda x: implementation_order.get(x, 1000))
    
    implementation_names = {
        'lsquic': 'LSQUIC',
        'quiche': 'quiche'
    }

        
    colors = [ColorDarkerBlue, ColorGreen, ColorRed]
    
    width, height = set_size('single')
    
    fig, ax1 = plt.subplots(figsize=(width, height*0.8))
    
    width = 0.3      
    
    N = len(lable)
    ind = np.arange(N)  # the x locations for the groups
    width = 0.35       # the width of the bars

    
    i = 0
    for idx, imp in enumerate(implementations):    
        to_plot = df.groupby(by=["server", "client"],as_index=False)[lable].median()
        d = to_plot.loc[(to_plot.server == imp) & (to_plot.client == imp)]
        d = d[lable].to_numpy()
        bp1 = ax1.bar(ind+i*width, d[0], width, color=colors[idx])
        i += 1
    
    ax1.set_xticks(ind + width / 2, )
    ax1.set_xticklabels(categories,rotation=15,ha='center')
    ax1.autoscale()
        
    ax1.set_xlabel(f"Categories {host}")
    ax1.set_ylabel('Samples (\%)')
    ax1.set_ylim(0, 80)
    #ax1.set_xscale('log')

    
    legend_elements = [
        Patch(facecolor=colors[i], edgecolor=colors[i], label=implementation_names[imp])
        for i, imp in enumerate(implementations)
    ]
    
    
    ax1.legend(handles=legend_elements, loc="upper right", facecolor='white', framealpha=1, ncol=1)
    fig.tight_layout()
    
    return fig, ax1


In [ ]:
perf_mixed = import_csv('../../results/perf/goracle/optimized/processed.csv', group=False)

In [ ]:
perf_mixed = perf_mixed.merge(perf_mixed.client_perf.apply(extend_perf_client), left_index=True, right_index=True)
perf_mixed = perf_mixed.merge(perf_mixed.server_perf.apply(extend_perf_server), left_index=True, right_index=True)

In [ ]:
fig, ax = plot_perf(perf_mixed, "server")
ax.set_xlabel(None)
fig.savefig('perf_categories.pdf')

In [ ]:
plot_perf(perf_mixed, "client")